In [16]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/results_avarages.csv" /tmp/results_avarages.csv
!ls /tmp/results_avarages.csv


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/tmp/results_avarages.csv


In [17]:
import pandas as pd
import altair as alt
import numpy as np

import plotly.graph_objects as go
from scipy.interpolate import griddata

In [18]:
df = pd.read_csv('/tmp/results_avarages.csv')
df

,ID,Width,Height,Depth,TemporalPooling,Epochs,Seed,SpatialPooling,AvgClassificationRate,AvgDeviation
0,17466591793,3,3,2,2,800,13,8,67.6382,5.2727
1,17466591793,3,3,2,2,800,42,8,65.6518,6.0814
2,17464856923,3,3,2,2,800,44,8,64.2906,4.6967
3,17466591793,3,3,2,2,800,45,8,67.1416,5.2558
4,17464856923,3,3,2,2,800,59,8,68.4826,5.7901
...,...,...,...,...,...,...,...,...,...,...
292,17466956263,7,7,5,3,800,59,8,62.0123,8.5894
293,17465580233,7,7,5,3,800,6,8,62.8351,4.8720
294,17465580233,7,7,5,3,800,93,8,63.2450,6.4720
295,17465580233,7,7,5,3,800,96,8,62.9333,5.4579


In [19]:
chart = alt.Chart(df).mark_boxplot().encode(
    x='Width:N',
    y='AvgClassificationRate:Q',
    color='Width:N'
).properties(
    title='Distribution of AvgClassificationRate by Filter Size'
)
chart

alt.Chart(...)

In [20]:
chart = alt.Chart(df).mark_boxplot().encode(
    x='TemporalPooling:N',
    y='AvgClassificationRate:Q',
    color='TemporalPooling:N'
).properties(
    title='Distribution of AvgClassificationRate by TemporalPooling'
)
chart

alt.Chart(...)

In [10]:
chart = alt.Chart(df).mark_boxplot().encode(
    x='Depth:N',
    y='AvgClassificationRate:Q',
    color='Depth:N'
).properties(
    title='Distribution of AvgClassificationRate by Depth'
)
chart

alt.Chart(...)

In [12]:
chart = alt.Chart(df).mark_rect().encode(
    x=alt.X('Depth:O'),
    y=alt.Y('TemporalPooling:N'),
    color=alt.Color('mean(AvgClassificationRate):Q', scale=alt.Scale(scheme='viridis')),
    tooltip=['Depth', 'TemporalPooling', 'mean(AvgClassificationRate)']
).properties(
    title='Heatmap of AvgClassificationRate by Depth and TemporalPooling'
)

chart

alt.Chart(...)

In [13]:
chart = alt.Chart(df).mark_rect().encode(
    x=alt.X('Width:O'),
    y=alt.Y('Depth:O'),
    color=alt.Color('mean(AvgClassificationRate):Q', scale=alt.Scale(scheme='viridis')),
    tooltip=['Width', 'Depth', 'TemporalPooling', 'mean(AvgClassificationRate)']
).facet(
    column='TemporalPooling:N'
).properties(
    title='AvgClassificationRate by Width, Depth, and TemporalPooling'
)

chart

alt.FacetChart(...)

In [14]:
# Create a grid
grid_x, grid_y, grid_z = np.mgrid[
    df['Width'].min():df['Width'].max():50j,
    df['Depth'].min():df['Depth'].max():50j,
    df['TemporalPooling'].min():df['TemporalPooling'].max():50j
]

# Prepare data
points = df[['Width', 'Depth', 'TemporalPooling']].values
values = df['AvgClassificationRate'].values

# Interpolate classification rates over the grid
grid_values = griddata(points, values, (grid_x, grid_y, grid_z), method='linear')

# Initialize the figure
fig = go.Figure()

# Add slices along TemporalPooling axis
for i in range(0, grid_values.shape[2], 10):
    fig.add_trace(go.Surface(
        z=grid_z[:, :, i],
        x=grid_x[:, :, i],
        y=grid_y[:, :, i],
        surfacecolor=grid_values[:, :, i],
        colorscale='Viridis',
        showscale=(i == 0),  # Show the scale only once, for the first surface
        colorbar=dict(title='AvgClassificationRate')
    ))

# Layout with labeled axes
fig.update_layout(
    title='3D Surface Plot of AvgClassificationRate',
    scene=dict(
        xaxis_title='Width',
        yaxis_title='Depth',
        zaxis_title='TemporalPooling'
    ),
    margin=dict(l=0, r=0, b=0, t=30)
)

fig.show()


In [15]:
# Filter TemporalPooling == 4
df_filtered = df[df['TemporalPooling'] == 4]

# Create a grid of Width x Depth (since TemporalPooling fixed)
width_grid = np.array([3, 5, 7])
depth_grid = np.array([2, 3, 4])
W, D = np.meshgrid(width_grid, depth_grid)

# Interpolate classification rates on grid
points = df_filtered[['Width', 'Depth']].values
values = df_filtered['AvgClassificationRate'].values

# Use 'nearest' to avoid NaNs in small discrete space
grid_rates = griddata(points, values, (W, D), method='nearest')

# Create 3D scatter plot
fig = go.Figure()

# Scatter points from your original data
fig.add_trace(go.Scatter3d(
    x=df_filtered['Width'],
    y=df_filtered['Depth'],
    z=df_filtered['AvgClassificationRate'],
    mode='markers',
    marker=dict(size=8, color=df_filtered['AvgClassificationRate'], colorscale='Viridis', colorbar=dict(title='AvgClassificationRate'))
))

# Surface plot from the interpolated grid (for visualization)
fig.add_trace(go.Surface(
    x=W,
    y=D,
    z=grid_rates,
    colorscale='Viridis',
    opacity=0.5,
    showscale=False
))

fig.update_layout(scene=dict(
    xaxis_title='Width',
    yaxis_title='Depth',
    zaxis_title='AvgClassificationRate'
))

fig.show()